In [27]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pprint
import csv
import datetime
import math


path = path = "/home/shenoy/Documents/Nikhil/research/enmd-pattern-testing/old/nikhil_postmortem_analyses/strong_scaling/temp/"


In [28]:
def extract_time(t1,t2,t3=None,t4=None):
    
    if isinstance(t1,float) and isinstance(t2,float) and isinstance(t3,float) and isinstance(t4,float):
        return (t2-t1)+(t4-t3)

    if isinstance(t1,float) and isinstance(t2,float) and t3 is None and t4 is None:
        return t2-t1
    
    if isinstance(t1,basestring):
        try:
            t1 = datetime.datetime.strptime(t1, "%Y-%m-%d %H:%M:%S.%f")
        except:
            t1 = datetime.datetime.strptime(t1, "%Y-%m-%d %H:%M:%S")
    
    if isinstance(t2,basestring):
        try:
            t2  = datetime.datetime.strptime(t2, "%Y-%m-%d %H:%M:%S.%f")
        except:
            t2  = datetime.datetime.strptime(t2, "%Y-%m-%d %H:%M:%S")

    if t3 is not None and isinstance(t3,basestring):
        
        try:
            t3   = datetime.datetime.strptime(t3, "%Y-%m-%d %H:%M:%S.%f")
        except:
            t3   = datetime.datetime.strptime(t3, "%Y-%m-%d %H:%M:%S")
    
    if t4 is not None and isinstance(t4,basestring):
        try:
            t4  = datetime.datetime.strptime(t4, "%Y-%m-%d %H:%M:%S.%f")
        except:
            t4  = datetime.datetime.strptime(t4, "%Y-%m-%d %H:%M:%S")
            
            
        return ((t2-t1)+(t4-t3)).total_seconds()
    
    else:
        if t3 is not None and t4 is not None:
            return ((t2-t1)+(t4-t3)).total_seconds()
        else:        
            return (t2-t1).total_seconds()

In [34]:
df = pd.read_csv(path + 'profile_128_0.csv',header=0,sep=',')
print df

           uid   iter  step           uid        step.1    Scheduling  \
0  unit.000022      1   sim  1.456696e+09  1.456696e+09  1.456696e+09   
1  unit.000023      1   ana  1.456696e+09  1.456696e+09  1.456696e+09   

    StagingInput   AgentStagingInputPending   AgentStagingInput  \
0   1.456696e+09               1.456696e+09        1.456696e+09   
1   1.456696e+09               1.456696e+09        1.456696e+09   

    AllocatingPending    Allocating   ExecutingPending     Executing  \
0        1.456696e+09  1.456696e+09       1.456696e+09  1.456696e+09   
1        1.456696e+09  1.456696e+09       1.456696e+09  1.456696e+09   

    AgentStagingOutputPending   AgentStagingOutput   PendingOutputStaging  \
0                1.456696e+09         1.456696e+09           1.456696e+09   
1                1.456696e+09         1.456696e+09           1.456696e+09   

    StagingOutput   Done  
0             NaN    NaN  
1             NaN    NaN  


In [30]:
def extract_timing_info(df,cores,inst):
    
    sim_data,ana_data = extract_data_time(df,cores,inst)
    sim_exec,ana_exec = extract_exec_time(df,cores,inst)
    
    return [sim_data,ana_data],[sim_exec,ana_exec]

In [31]:
def extract_data_time(df,cores,inst):
    df
    req_df = df[['step','StagingInput','Allocating','AgentStagingOutputPending','Done']]
    
    #preloop_df = req_df[:1]
    sim_df = req_df[1:-2]
    ana_df = req_df[-2:]
    
    #preloop_data = None
    sim_data = None
    ana_data = None
    
    
    # Preloop extraction
    #step,t1,t2,t3,t4 =  preloop_df.loc[0,:]
    #preloop_data = extract_time(t1,t2,t3,t4)
    
    
    #Simulation extraction
    per_unit_data_time  = list()
    for row in sim_df.iterrows():
        step,t1,t2,t3,t4 = row[1:][0]
        #print t1,t2,t3,t4
        #print row
        per_unit_data_time.append(extract_time(t1,t2,t3,t4))
        
    if cores == inst:
        kern_1 = per_unit_data_time[:inst]
        kern_2 = per_unit_data_time[inst:]
        sim_data = np.average(kern_1) + np.average(kern_2)
    elif inst > cores:
        pass
    
    
    #Analysis extraction
    per_unit_data_time  = list()
    for row in ana_df.iterrows():
        step,t1,t2,t3,t4 = row[1:][0]
        #print t1,t2,t3,t4
        #print row
        per_unit_data_time.append(extract_time(t1,t2,t3,t4))
        
    ana_data = sum(per_unit_data_time)
    
    #print preloop_data, sim_data, ana_data
    return sim_data, ana_data

In [32]:
def extract_exec_time(df,cores,inst):
    req_df = df[['step','Executing','AgentStagingOutputPending']]
    
    #preloop_df = req_df[:1]
    sim_df = req_df[1:-2]
    ana_df = req_df[-2:]
    
    #preloop_exec = None
    sim_exec = None
    ana_exec = None
    
    
    # Preloop extraction
    #step,t1,t2 =  preloop_df.loc[0,:]
    #preloop_exec = extract_time(t1,t2)

    
    #Simulation extraction
    #STUPID ME SET CORES PER CU = 2!!
    per_unit_exec_time  = list()
    for row in sim_df.iterrows():
        step,t1,t2 = row[1:][0]
        #print t1,t2,t3,t4
        #print row
        per_unit_exec_time.append(extract_time(t1,t2))
        
    if cores == inst:
        kern_1 = per_unit_exec_time[:inst]
        kern_2 = per_unit_exec_time[inst:]
        sim_exec = (np.average(kern_1) + np.average(kern_2))*2
    elif inst > cores:
        pass
    
  
    
    #Analysis extraction
    per_unit_exec_time  = list()
    for row in ana_df.iterrows():
        step,t1,t2 = row[1:][0]
        #print t1,t2,t3,t4
        #print row
        per_unit_exec_time.append(extract_time(t1,t2))
        
    ana_exec = sum(per_unit_exec_time)
    
    
    #print preloop_exec, sim_exec, ana_exec
    return sim_exec, ana_exec

In [33]:
data, exec_list = extract_timing_info(df,cores=128,inst=128)

KeyError: "['step' 'StagingInput' 'Allocating' 'AgentStagingOutputPending' 'Done'] not in index"